In [ ]:
import json 
import re

path = "training_data/lastfm"
# suffix_list = ['', '_CoT', '_ToT', '_IRSfix', '_IRSori', '_POP', '_random']
# _suf = suffix_list[5]
suffix_list = ['_IRSfix', '_IRSori', '_POP']
 
from openai import OpenAI

client = OpenAI(
    api_key=''
)

prompt_system_accept = "You are a recommender system. Given the user profile and historical data, analyze the user's interests. Based on this information, would the user be interested in the music artists in the Influence path step by step? Answer with a probability for an artist between 0 and 1, where 0 means 'definitely not interested' and 1 means 'definitely interested'. If uncertain, make your best guess, then return the mean probability."

prompt_system_relevance = "You are a professional music critic. Given the Influence path, based on your understanding of the music artists, what's the relatedness of each 2 adjacent artists in the influence path? Answer with a probability between 0 and 1, where 1 means 'definitely related' and 0 means 'definitely not related'. If uncertain, make your best guess, then return the mean probability."

# prompt_list = [prompt_system_accept, prompt_system_relevance]
# prompt_suf = ['_accept', '_rel']

system_prompt_list = [[prompt_system_accept, '_accept'], [prompt_system_relevance, '_rel']]

def get_score(_question, _prompt, _system):
    # influence_path = get_movies(_question)
    
    if _prompt == []:
        return [0,0]
    else:
        influence_path = str.join("\n", _prompt)

    user_prompt = f"Historical data:\n{_question}\nInfluence path:\n{influence_path}"
    
    # print(user_prompt)
    
    
    loop_flag = 0
    while True:
        message = [
            {"role": "system", "content": _system},
            {"role": "user", "content": user_prompt}
        ]
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
                messages=message
            )
        # print(completion.choices[0].finish_reason)
        message.append({"role": "assistant", "content": completion.choices[0].message.content})
        message.append({"role": "user", "content": "Only numerical response required, without any other word. Output the mean probability."})
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=message
        )
        
        gpt_reply = completion.choices[0].message.content
        # print(gpt_reply)
        # print(gpt_reply)
        # print(mean_pro)
        if re.search('0\.\d+', gpt_reply):
            return [float(re.search('0\.\d+', gpt_reply).group()), gpt_reply]
        
        loop_flag += 1
        if loop_flag >= 2:
            print(gpt_reply)
            
for _prompt, _system_suf in system_prompt_list:
    print(_system_suf)
    for _suf in suffix_list:
        with open(f"training_data/lastfm/output_gpt_lf{_suf}_pro.json", "r", encoding='utf-8') as f:
            path_result = json.load(f)
        with open("training_data/train_lf.json", "r", encoding='utf-8') as f:
            training_data_prompt = json.load(f)
    
        k = 1
        all_result = []
        for his_tra, influ_path in zip(training_data_prompt['llm'], path_result):
            # if(k%20==0):
            #     print(k)
            print(k)
            k+=1
            all_result.append([get_score(his_tra[1], influ_path[0], _prompt),get_score(his_tra[1], influ_path[1], _prompt)])
            # print(all_result)
            # break
            if k == 20:
                break
        
        with open(f"training_data/lastfm/llm_result_{_suf}{_system_suf}_GPT3.json", "w") as outfile:
            json.dump(all_result, outfile)
        sum = 0
        count = 0
        
        for _ in all_result:
            a = _[0][0]
            b = _[1][0]
            max_ = max(a, b)
            if max_ == 0:
                continue
            sum += max_
            count += 1
        
        print(_suf)
        print(sum/count)